In [ ]:
import pandas as pd
import numpy as np
import random

# Parâmetros
n = 10000  # Volume realista e útil para mapas de calor
data_inicio = pd.to_datetime('2025-11-15 15:00:00')  # 3h antes do evento
data_fim = pd.to_datetime('2025-11-16 05:00:00')  # 3h depois do evento

# Datas em intervalos regulares
datas = pd.date_range(data_inicio, data_fim, periods=n)

# Localizações dos sensores
locais = ['Norte', 'Sul', 'Leste', 'Oeste', 'Centro']

# Gerando os dados
data = {
    'data_coleta': datas,
    'nivel_ruido_db': np.random.normal(loc=65, scale=10, size=n),  # média 65 dB
    'poluente_pm25': np.random.normal(loc=35, scale=15, size=n),  # média 35 µg/m³
    'densidade_pessoas_m2': np.clip(np.random.normal(loc=2, scale=1, size=n), 0, None),
    'sensor_localizacao': np.random.choice(locais, size=n)
}

df = pd.DataFrame(data)

# Campo calculado: alerta
df['alerta'] = (df['nivel_ruido_db'] > 80) | (df['densidade_pessoas_m2'] > 4)

# Ajustando formatos
df['nivel_ruido_db'] = df['nivel_ruido_db'].round(2)
df['poluente_pm25'] = df['poluente_pm25'].round(2)
df['densidade_pessoas_m2'] = df['densidade_pessoas_m2'].round(2)

# Exportando para CSV
df.to_csv('sensores_urbanos.csv', index=False)

print("✅ Dataset 'sensores_urbanos.csv' gerado com sucesso.")

Saída da execução
0KB
	Stream
		✅ Dataset 'sensores_urbanos.csv' gerado com sucesso.




In [2]:
import pandas as pd

# Convertendo timestamp para datetime
df_sensor['timestamp'] = pd.to_datetime(df_sensor['timestamp'])

### =====================================
### 1. Dados para o Mapa de Calor de Aglomerações por Localização
### =====================================

# Filtra apenas registros com aglomeração True
df_aglomeracao = df_sensor[df_sensor['aglomeração'] == True]

# Conta número de aglomerações por local
df_mapa_calor = df_aglomeracao.groupby('localizacao_camera').size().reset_index(name='qtd_aglomeracoes')

print("Mapa de Calor - Aglomerações por Localização")
print(df_mapa_calor)

### =====================================
### 2. Dados para a Série Temporal - Evolução das Aglomerações
### =====================================

# Cria uma coluna de timestamp arredondada por hora
df_sensor['timestamp_hora'] = df_sensor['timestamp'].dt.floor('H')

# Agrega quantidade de eventos True por hora
df_series_temporal = df_sensor.groupby('timestamp_hora').agg({
    'aglomeração': 'sum',
    'veiculo_parado': 'sum',
    'pessoa_caida': 'sum'
}).reset_index()

print("\nSérie Temporal - Evolução das Aglomerações e outros eventos")
print(df_series_temporal)

### =====================================
### 3. Dados para o Pie Chart - Proporção de Eventos Registrados
### =====================================

# Soma total de ocorrências para cada evento
evento_counts = {
    'Aglomeração': df_sensor['aglomeração'].sum(),
    'Veículo parado': df_sensor['veiculo_parado'].sum(),
    'Pessoa caída': df_sensor['pessoa_caida'].sum()
}

# Transforma em DataFrame
df_pie_chart = pd.DataFrame(list(evento_counts.items()), columns=['evento', 'qtd_ocorrencias'])

print("\nPie Chart - Proporção de Eventos Registrados")
print(df_pie_chart)


Mapa de Calor - Aglomerações por Localização
  localizacao_camera  qtd_aglomeracoes
0           Camera_1              3185
1           Camera_2              3179
2           Camera_3              3184
3           Camera_4              3178
4           Camera_5              3178

Série Temporal - Evolução das Aglomerações e outros eventos
        timestamp_hora  aglomeração  veiculo_parado  pessoa_caida
0  2025-11-15 15:00:00         1031             432           124
1  2025-11-15 16:00:00         1104             373            81
2  2025-11-15 17:00:00         1004             305           154
3  2025-11-15 18:00:00         1105             389           299
4  2025-11-15 19:00:00         1278             307            83
5  2025-11-15 20:00:00         1232             395           126
6  2025-11-15 21:00:00          938             454           225
7  2025-11-15 22:00:00         1144             376           114
8  2025-11-15 23:00:00         1060             520           206


<ipython-input-2-6cdb35c59760>:27: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_sensor['timestamp_hora'] = df_sensor['timestamp'].dt.floor('H')


In [3]:
df_sensor.to_csv('sensor_urbano_novo.csv', index=False)

# Para baixar o arquivo:
from google.colab import files
files.download('sensor_urbano_novo.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>